# Task 2 - Train on the NEU train after Histogram Equalization


In [1]:
import os
import numpy as np

In [2]:
train_crazing_dir = os.path.join('data/NEU-DET-HIS/train/images/crazing')
train_inclusion_dir = os.path.join('data/NEU-DET-HIS/train/images/inclusion')
train_patches_dir = os.path.join('data/NEU-DET-HIS/train/images/patches')
train_pitted_surface_dir = os.path.join('data/NEU-DET-HIS/train/images/pitted_surface')
train_rolledin_scale_dir = os.path.join('data/NEU-DET-HIS/train/images/rolled-in_scale')
train_scratches_dir = os.path.join('data/NEU-DET-HIS/train/images/scratches')

In [3]:
train_crazing_names = os.listdir(train_crazing_dir)
train_inclusion_names = os.listdir(train_inclusion_dir)
train_patches_names = os.listdir(train_patches_dir)
train_pitted_surface_names = os.listdir(train_pitted_surface_dir)
train_rolledin_scale_names = os.listdir(train_rolledin_scale_dir)
train_scratches_names = os.listdir(train_scratches_dir)

In [4]:
print('total training crazing images:', len(train_crazing_names))
print('total training inclusion images:', len(train_inclusion_names))
print('total training patches images:', len(train_patches_names))
print('total training pitted_surface images:', len(train_pitted_surface_names))
print('total training rolled_in_scale images:', len(train_rolledin_scale_names))
print('total training scratches images:', len(train_scratches_names))

total training crazing images: 241
total training inclusion images: 241
total training patches images: 241
total training pitted_surface images: 241
total training rolled_in_scale images: 241
total training scratches images: 240


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.Sequential([
                                    # First Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Second Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Third Convolution
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    tf.keras.layers.Dense(6, activation='softmax'),])


In [7]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 32)       0

In [8]:
from tensorflow.keras.optimizers import RMSprop, Adam

model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Rescaling images
data_generator = ImageDataGenerator(rescale=1/255)

train_generator = data_generator.flow_from_directory(
    directory = 'data/NEU-DET-HIS/train/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    directory = 'data/NEU-DET-HIS/validation/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'

)

Found 1440 images belonging to 6 classes.
Found 360 images belonging to 6 classes.


In [11]:
# Load the TensorBoard notebook extension
import datetime

%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_generator,steps_per_epoch=8,epochs=5, verbose=1,callbacks=[tensorboard_callback])
%tensorboard --logdir logs/fit

Epoch 1/5
8/8 [==============================] - 27s 3s/step - loss: 2.1546 - accuracy: 0.1479
Epoch 2/5
8/8 [==============================] - 19s 2s/step - loss: 1.7488 - accuracy: 0.2271
Epoch 3/5
8/8 [==============================] - 18s 2s/step - loss: 1.5491 - accuracy: 0.3417
Epoch 4/5
8/8 [==============================] - 18s 2s/step - loss: 1.4062 - accuracy: 0.4167
Epoch 5/5
8/8 [==============================] - 19s 2s/step - loss: 0.9840 - accuracy: 0.6208


Reusing TensorBoard on port 6006 (pid 12072), started 1 day, 22:22:17 ago. (Use '!kill 12072' to kill it.)

In [12]:
history.history

{'loss': [2.154602289199829,
  1.7488042116165161,
  1.5491185188293457,
  1.4061915874481201,
  0.9840208888053894],
 'accuracy': [0.14791665971279144,
  0.22708334028720856,
  0.34166666865348816,
  0.4166666567325592,
  0.6208333373069763]}

## Test on Test on NEU test, and whole ENEU after Histogram Equalization

In [13]:
tf.config.run_functions_eagerly(True)
model.evaluate(validation_generator)

C:\Users\91800\anaconda3\envs\MTP\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 [==============================] - 5s 788ms/step - loss: 0.8964 - accuracy: 0.5167


[0.8964059352874756, 0.5166666507720947]

In [18]:
validation_generator_ENEU = data_generator.flow_from_directory(
    directory = 'data/ENEU-HIS/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical'
)

Found 10800 images belonging to 6 classes.


In [19]:
model.evaluate(validation_generator_ENEU)

180/180 [==============================] - 140s 776ms/step - loss: 2.7253 - accuracy: 0.2051


[2.725252866744995, 0.20509259402751923]